# Import Library

This code is for importing library what I need.

In [1]:
!pip install numpy
!pip install pandas==1.5.3
!pip install nltk
!pip install beautifulsoup4
!pip install mpld3
!pip install -U scikit-learn
!pip install joblib
!pip install gensim
!pip install pyLDAvis==3.4.0

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Not uninstalling pandas at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'pandas'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 KB 20.8 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 92.5 MB/s eta 0:00:00
 

In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup
import re
import os 
import codecs
from sklearn import feature_extraction
import mpld3
from glob import glob 
import json

import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-05 07:47:23.231752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 07:47:23.427745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: ca

# Document Clustering

### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the titles/bodies. First, I load NLTK's list of English stop words. Stop words are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [3]:
# load nltk's English stopwords as variable called 'stopwords'
new_stopwords = ["use", "abl", "would", "like", "made", "happen", "id", "n\'t", "expect", "1", "useline", "arn", "aw", "int2byt", "py35", "python", "kw", "def", "six", "six.mov", "\'s", "ns", "-r", "int2byte_alt", "python3", "six.add_metaclass", "setup.pi", "import_modul", "r", "x", "__name__", "0m", "1m" ,"\'m", "'m", "flask", "pkgutil.get_load", "ani", "-g", "_configtest.c", "unknown._", "f", "\'d", "|sin", "|cos", "h", "numpy.float64", "|squar", "|absolut", "|sqrt", "|reciproc", "b", "-fwrapv", "defin", "x86_64-linux-gnu-gcc", "-fstack-protector-strong", "numpi", "µs", "ms", "//projects.scipy.org/numpy/ticket/709", "v", "datetime.datetime.now", "side=\'left", "bench_avx", "v_shape", "np.searchsort", "__init__.pi", "array_lik", "ss", "cdk", "aws-cdk", "kwarg", "name", "iam", "aws-cdk/cor", "aws-cdk/cfnspec", "callabl", "arg", "l", "0x485b9e", "0x0", "cjw296", "numpy.isnan", "xxx", "\\n", "__init__"]
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(new_stopwords)
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Next I import the Snowball Stemmer which is actually part of NLTK. Stemming is just the process of breaking a word down into its root.

In [4]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

Below I define two functions: 
tokenize_and_stem: tokenizes (splits the titles into a list of its respective words (or tokens) and also stems each token tokenize_only: tokenizes the titles only I use both these functions to create a dictionary which becomes important in case I want to use stems for an algorithm, but later convert stems back to their full words for presentation purposes. Guess what, I do want to do that!

In [5]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Latent Dirichlet Allocation

For my implementaiton of LDA, I use the Gensim package. I'm going to preprocess the titles a bit differently here, and first I define a function to remove any proper noun.

In [6]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [7]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [8]:
#Latent Dirichlet Allocation implementation with Gensim

from gensim import corpora, models, similarities 

## Title and Body Newcomers Scenario

### Stopwords, stemming, and tokenizing

Access, split and compute data from newcomers body and title txt files.

In [9]:
#import data title
titles = open('data_title_newcomers.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
titles = titles[1:1826]

#import data body
bodies = open('data_body_newcomers.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
bodies = bodies[1:1826]

print(str(len(titles)) + ' titles')
print(str(len(bodies)) + ' bodies')

37 titles
37 bodies


Enter newcomers body and title data into the list

In [10]:
# generates index for each item
list_titles = []

for text in titles:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_titles.append(text)

titles = list_titles

# generates index for each item
list_bodies = []

for text in bodies:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_bodies.append(text)

bodies = list_bodies

bodies_titles = []

for i in range(len(bodies)):
    item = bodies[i] + titles[i]
    bodies_titles.append(item)

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Check newcomers body and title data.

In [11]:
pd.DataFrame(bodies_titles)

,0
0,https://github.com/pallets/flask/blob/9039534e...
1,"Just a small contribution, in documentation se..."
2,The following two lines:\nhttps://github.com/p...
3,The following line contains an unused import:\...
4,A bunch of properties in the codebase are defi...
5,`Blueprint` does not have reStructuredText mar...
6,Python can be compiled without certain modules...
7,---\n\n### Expected Behavior\n\n`yum install p...
8,The `flask.testing` `FlaskClient` has clean-up...
9,### Expected Behavior\n\nExamples in the docum...


Below I use my stemming/tokenizing and tokenizing functions to iterate over the list of bodies and titles to create two vocabularies: one stemmed and one only tokenized.

In [12]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in bodies_titles:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

Using these two lists, I create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column. The benefit of this is it provides an efficient way to look up a stem and return a full token. The downside here is that stems to tokens are one to many: the stem 'run' could be associated with 'ran', 'runs', 'running', etc. For my purposes this is fine--I'm perfectly happy returning the first token associated with the stem I need to look up.

In [13]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

Check newcomers bodies and titles data vocab frame.

In [14]:
vocab_frame

,words
https,https
//github.com/pallets/flask/blob/9039534eee6a87da98a1dee9e4338d1b73e861f8/src/flask/ctx.pi,//github.com/pallets/flask/blob/9039534eee6a87...
l133,l133
while,while
it,it
...,...
templat,template
lookup,lookup
not,not
document,documented


Check the shape of items in vocab_frame

In [15]:
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 2591 items in vocab_frame


### Latent Dirichlet Allocation

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [16]:
#Latent Dirichlet Allocation implementation with Gensim

#remove proper names
preprocess = [strip_proppers(doc) for doc in bodies_titles]

%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

CPU times: user 54.3 ms, sys: 0 ns, total: 54.3 ms
Wall time: 56.4 ms
CPU times: user 7.48 ms, sys: 0 ns, total: 7.48 ms
Wall time: 7.43 ms


In [17]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

40


Below are some Gensim specific conversions; I also filter out extreme words (see inline comment)

In [18]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
len(corpus)

37

The actual model runs below. I took 1825 passes to ensure convergence, but you can see that it took my machine 4 minutes to run. My chunksize is larger than the corpus so basically all synopses are used per pass. I should optimize this, and Gensim has the capacity to run in parallel. I'll likely explore this further as I use the implementation on larger corpora.

In [20]:
# %time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, update_every=5, chunksize=10000, passes=100)
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

CPU times: user 2.08 s, sys: 2.88 ms, total: 2.09 s
Wall time: 2.1 s


Each topic has a set of words that defines it, along with a certain probability.

In [21]:
# lda.show_topics()

pprint(lda.print_topics())

[(0,
  '0.022*"import" + 0.014*"test" + 0.012*"pip" + 0.012*"easy_instal" + '
  '0.010*"context" + 0.010*"autocorrect" + 0.010*"datetim" + 0.010*"doc" + '
  '0.010*"https" + 0.008*"exampl"'),
 (1,
  '0.025*"document" + 0.018*"mode" + 0.015*"https" + 0.013*"doc" + '
  '0.010*"link" + 0.010*"develop" + 0.010*"read" + 0.010*"sure" + '
  '0.010*"secur" + 0.010*"extens"'),
 (2,
  '0.024*"instal" + 0.019*"yum" + 0.019*"env" + 0.015*"version" + '
  '0.015*"virtualenv" + 0.015*"python-virtualenv" + 0.015*"static" + '
  '0.010*"instead" + 0.010*"exampl" + 0.010*"problem"'),
 (3,
  '0.019*"file" + 0.019*"test" + 0.019*"mode" + 0.017*"contain" + '
  '0.017*"object" + 0.017*"zero-pad" + 0.014*"warn" + 0.014*"tree" + '
  '0.014*"url" + 0.014*"properti"'),
 (4,
  '0.027*"client" + 0.022*"request" + 0.017*"http" + 0.014*"document" + '
  '0.011*"test" + 0.011*"empti" + 0.011*"string" + 0.011*"werkzeug" + '
  '0.011*"session" + 0.011*"side"')]


In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.032274 -0.046194       1        1  29.798899
1     -0.133752 -0.047673       2        1  22.291693
4      0.003139  0.157354       3        1  20.212415
3      0.118793 -0.047957       4        1  20.043155
2     -0.020455 -0.015529       5        1   7.653839, topic_info=         Term       Freq      Total Category  logprob  loglift
132    instal   5.000000   5.000000  Default  30.0000  30.0000
167    client   8.000000   8.000000  Default  29.0000  29.0000
64     import   8.000000   8.000000  Default  28.0000  28.0000
354      mode  10.000000  10.000000  Default  27.0000  27.0000
15       file   6.000000   6.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
131  explicit   0.526664   1.748518   Topic5  -5.1965   1.3700
125      also   0.526631   1.771299   Topic5  -5.1966   1.3570
44       seem   0.526615   3.994817   Topic5  -5.1966   0.5437
471     field   0.526566   1.821177   Topic5  -5.1967   1.3291
426     think   0.526560   3.203770   Topic5  -5.1967   0.7642

[265 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
122       5  0.925867                                                 -p
152       3  0.763487                   //flask.pocoo.org/docs/0.12/api/
153       3  0.505476                //flask.pocoo.org/docs/1.0/testing/
314       2  0.750442             //flask.pocoo.org/docs/dev/reqcontext/
0         1  0.941809  //github.com/pallets/flask/blob/9039534eee6a87...
...     ...       ...                                                ...
452       4  0.754936                                               warn
586       3  0.904861                                           werkzeug
382       2  0.988205                                               work
151       5  0.834550                                                yum
453       4  0.862000                                           zero-pad

[325 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3])

In [23]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)#0.5588166880236373


Coherence Score:  0.38844973072360345


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=78a30958-227f-4a9f-a41b-19f0e30c3928' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>